In [1]:
import qecc as qc
import numpy as np
from termcolor import colored

Conventions: 

- X coords extend vertically |
- Z coords extend horizontally -


In [2]:
stabs = ['ZZIZZIZZI', 'IZZIZZIZZ', 'XXXXXXIII', 'IIIXXXXXX', 'ZZIIIIIII', 'IZZIIIIII', 'IIIIIIZZI', 'IIIIIIIZZ']
Zlog = ['ZZZZZZZZZ']
Xlog = ['XXXXXXXXX']
for s in stabs:
    print(len(s))
code = qc.StabilizerCode(stabs, Xlog, Zlog)

9
9
9
9
9
9
9
9


In [3]:
enc = next(code.encoding_cliffords())
print(enc)

X[0] |->  +X[0] X[1] X[2] X[3] X[4] X[5] X[6] X[7] X[8]
X[1] |->  +X[0] X[1] X[2] X[3]
X[2] |->  +X[3] X[4]
X[3] |->  +Z[5] Z[6]
X[4] |->  +Z[2] Z[3] Z[4] Z[5]
X[5] |->  +X[0] X[4] X[5] X[6] X[7] X[8]
X[6] |->  +X[0] X[1] X[3] X[4]
X[7] |->  +X[0] X[1] X[2] X[3] X[7] X[8]
X[8] |->  +X[0] X[1] X[2] X[5] X[6] X[7]
Z[0] |->  +Z[0] Z[1] Z[2] Z[3] Z[4] Z[5] Z[6] Z[7] Z[8]
Z[1] |->  +Z[0] Z[1] Z[3] Z[4] Z[6] Z[7]
Z[2] |->  +Z[1] Z[2] Z[4] Z[5] Z[7] Z[8]
Z[3] |->  +X[0] X[1] X[2] X[3] X[4] X[5]
Z[4] |->  +X[3] X[4] X[5] X[6] X[7] X[8]
Z[5] |->  +Z[0] Z[1]
Z[6] |->  +Z[1] Z[2]
Z[7] |->  +Z[6] Z[7]
Z[8] |->  +Z[7] Z[8]


In [4]:
def split(stab, split, ):
    """
    Split the input stabilizer at the given split operator 
    """
    
    

def insert_gauge(gin, stabs, gauge):
    """
    [0] Insert gin into stabilizer group,
    [1] update stabs to be lowest-weight generators
    [2] remove anticommuators from gauge group
    """
    gop = qc.Pauli(gin)
    
    # update stabilizer group
    for idx, s in enumerate(stabs):
        old_weight = qc.Pauli(s).wt
        new_weight = (qc.Pauli(s)*gop).wt
        if(new_weight < old_weight):
            stabs[idx] = (qc.Pauli(s)*gop).op
    stabs.append(gin)
        
    # update gauge group
    for idx, g in enumerate(gauge):
        if qc.com(qc.Pauli(g), gop):
            del gauge[idx]
    
    return [stabs, gauge]

In [5]:
class Lattice_2d():
    def __init__(self, dimX, dimZ):
        self._dimX = dimX
        self._dimZ = dimZ
        self._colors = ['I'] * (dimX-1)*(dimZ-1)
        self._stabs = self.bacon_shor_group()
        self._gauge = self.elem_gauge_group()
        
    def __str__(self):
        vertex_rows = []
        face_rows = []
        dimX = self._dimX
        dimZ = self._dimZ
        for i in range(dimX):
            vertex_string = ''
            for j in range(dimZ):
                vertex_string += str(i*dimZ + j).zfill(3)
                if (j != dimZ-1):
                    vertex_string += '---'
            vertex_rows.append(vertex_string)
                
        for i in range(dimX-1):
            face_string = ''
            for j in range(dimZ-1):
                if(self._colors[i*(dimZ-1) + j] == 'R'):
                    face_string += ' | '+colored(' # ', 'red')
                elif(self._colors[i*(dimZ-1) + j] == 'B'):
                    face_string += ' | '+colored(' # ', 'blue')
                elif(self._colors[i*(dimZ-1) + j] == 'I'):
                    face_string += ' |    '
                else:
                    raise ValueError(f'Invalid color type {self._colors[i*dimZ+j]}')
                if j == dimZ-2:
                    face_string += ' |'
            face_rows.append(face_string)
        sout = ''
        for idx, row in enumerate(vertex_rows):
            sout += row +'\n'
            if idx != len(vertex_rows)-1:
                sout += face_rows[idx]+'\n'
        return sout
        
    def bacon_shor_group(self):
        Xs = []
        Zs = []
        dimX = self._dimX
        dimZ = self._dimZ
        for i in range(dimZ-1):
            s = list('I'*dimZ*dimX)
            for j in range(dimX):
                s[j*dimZ+i] = 'Z'
                s[j*dimZ+i+1] = 'Z'
            Zs.append(''.join(s))
        for i in range(dimX-1):
            s = list('I'*dimX*dimZ)
            for j in range(dimZ):
                s[j+i*dimZ] = 'X'
                s[j+(i+1)*dimZ] = 'X'
            Xs.append(''.join(s))
        return [Xs, Zs]
    
    def elem_gauge_group(self):
        Xs = []
        Zs = []
        dimX = self._dimX
        dimZ = self._dimZ
        # make X-type gauge ops
        for i in range(dimZ):
            for j in range(dimX-1):
                s = list('I'*dimX*dimZ)
                s[i+j*dimZ] = 'X'
                s[i+(j+1)*dimZ] = 'X'
                Xs.append(''.join(s))
        # make Z-type gauge ops
        for i in range(dimX):
            for j in range(dimZ-1):
                s = list('I'*dimX*dimZ)
                s[i*dimZ+j] = 'Z'
                s[i*dimZ+j+1] = 'Z'
                Zs.append(''.join(s))
        return [Xs, Zs]
    
    def display(self, pauli):
        dimX = self._dimX
        dimZ = self._dimZ
        if (len(pauli) != dimX*dimZ):
            raise ValueError("Stabilizer dimension mismatch with lattice size")
        sout = ''
        slist = list(pauli)
        for i in range(dimX):
            for j in range(dimZ):
                if slist[i*dimZ+j] == 'X':
                    sout += ' X '
                elif slist[i*dimZ+j] == 'Z':
                    sout += ' Z '
                else:
                    sout += '   '
                if (j != dimZ-1):
                    sout += '---'
            if (i != dimX -1):
                sout += '\n'
                sout += ' |    '*dimZ
            sout += '\n'
        print(sout)
        
    
    def update_groups(self, coords, cut_type):
        """
        cut the stabilizer group by coloring the face with the given type
            AND
        update the gauge group 
    
        algo: 
        [0] pick the gauge operator g to cut around
        [1] find s \in S that has weight-2 overlap with g
        [2] divide that s 
        [3] update the gauge group 
        """
        (i, j) = coords
        dimX = self._dimX
        dimZ = self._dimZ
        [Sx, Sz] = self._stabs
        [Gx, Gz] = self._gauge
        
        
        
        if cut_type == 'R':
            # R is a Z-cut
            g = ['I'] * dimX*dimZ
            g[i*dimZ + j] = 'Z'
            g[i*dimZ + j + 1] = 'Z'
            
            gop = qc.Pauli(''.join(g))
            
            # cut the relevant stabilizer
            for idx, s in enumerate(Sz):
                # find the overlapping stabilizer
                if (qc.Pauli(s)*gop).wt == qc.Pauli(s).wt - 2:
                    # cut s into two vertical parts 
                    s1 = ['I'] * dimX*dimZ
                    s2 = ['I'] * dimX*dimZ
                    for k in range(0, i+1):
                        s1[k*dimZ + j] = s[k*dimZ + j]
                        s1[k*dimZ + j+1] = s[k*dimZ + j+1]
                    for k in range(i+1, dimZ):
                        s2[k*dimZ + j] = s[k*dimZ + j]
                        s2[k*dimZ + j+1] = s[k*dimZ + j+1]
                    del Sz[idx]
                    Sz.append(''.join(s1))
                    Sz.append(''.join(s2))
                    break
            
            # make new gauge operator and update gauge group 
            gauge = ['I'] * dimX*dimZ
            for k in range(0, j+1):
                gauge[i + k*dimZ] = 'Z'
                gauge[i + k*dimZ + 1] = 'Z'
            for idx, gx in enumerate(Gx):
                if qc.com(qc.Pauli(''.join(gx)), qc.Pauli(''.join(gauge))):
                    del Gx[idx]    
                
        elif cut_type == 'B':
            # B is a X-cut:
            g = ['I'] * dimX*dimZ
            g[i*dimZ + j] = 'X'
            g[(i+1)*dimZ + j ] = 'X'
            
            gop = qc.Pauli(''.join(g))
            
            # cut the relevant stabilizer
            for idx, s in enumerate(Sx):
                # find the overlapping stabilizer
                if (qc.Pauli(s)*gop).wt == qc.Pauli(s).wt - 2:
                    # cut s into two horizontal parts 
                    s1 = ['I'] * dimX*dimZ
                    s2 = ['I'] * dimX*dimZ
                    for k in range(0, j+1):
                        s1[i*dimZ + k] = s[i*dimZ + k]
                        s1[(i+1)*dimZ + k] = s[(i+1)*dimZ + k]
                    for k in range(j+1, dimX):
                        s2[i*dimZ + k] = s[i*dimZ + k]
                        s2[(i+1)*dimZ + k] = s[(i+1)*dimZ + k]
                    del Sx[idx]
                    Sx.append(''.join(s1))
                    Sx.append(''.join(s2))
                    break
            
            # make new gauge operator and update gauge group 
            gauge = ['I'] * dimX*dimZ
            for k in range(0, i+1):
                gauge[j + i*dimZ] = 'X'
                gauge[j + (i+1)*dimZ] = 'X'
            self.display(gauge)
            for idx, gx in enumerate(Gz):
                if qc.com(qc.Pauli(''.join(gx)), qc.Pauli(''.join(gauge))):
                    del Gz[idx]
                    

        # update the groups
        self._stabs = [Sx, Sz]
        self._gauge = [Gx, Gz]
        
        
        
    def color_face(self, coords, cut_type):
        (x, z) = coords
        if (x >= self._dimX-1) or (z >= self._dimZ-1):
            raise ValueError(f'Face coordinant ({x}, {z}) out of lattice range')
        self._colors[x*(self._dimZ-1) + z] = cut_type
        self.update_groups(coords, cut_type)

In [6]:
def make_encoder(dims, stabs, gauges):
    """
    Make the encoding matrix of a code
    
    We find the Clifford operator based on a logical X and Z
        that are cuts across the first row and column
    """
    [dimX, dimY] = dims
    [Sx, Sz] = stabs
    [Gx, Gz] = gauges
    
    # make logical operators
    Lx = ['I']*dimX*dimZ
    Lz = ['I']*dimX*dimZ
    for j in range(dimZ):
        Lx[j] = 'X'
    for i in range(dimX):
        Lz[i*dimZ] = 'Z'
    Lx = ''.join(Lx)
    Lz = ''.join(Lz)
    
    # make constraints
    # that is the Z-logical, stabs, and n-1-s Z-gauges
    constraints_in = [Lz]+Sz+Sx
    for k in range(dimX*dimZ-len(constraints_in)):
        constraints_in.append(Gz[k])
    
    # find clifford operator 
    # that maps 
    

In [7]:
lat = Lattice_2d(3,3)
lat.color_face((1, 1),'R')
lat.color_face((0, 0),'B')
[Sx, Sz] = lat._stabs
[Gx, Gz] = lat._gauge
print(lat)

 X ---   ---   
 |     |     |    
 X ---   ---   
 |     |     |    
   ---   ---   

000---001---002
 |  #  |     |
003---004---005
 |     |  #  |
006---007---008



In [8]:
for s in Sz:
    print(s)
    lat.display(s)

ZZIZZIZZI
 Z --- Z ---   
 |     |     |    
 Z --- Z ---   
 |     |     |    
 Z --- Z ---   

IZZIZZIII
   --- Z --- Z 
 |     |     |    
   --- Z --- Z 
 |     |     |    
   ---   ---   

IIIIIIIZZ
   ---   ---   
 |     |     |    
   ---   ---   
 |     |     |    
   --- Z --- Z 



In [9]:
for g in Gx:
    lat.display(g)

 X ---   ---   
 |     |     |    
 X ---   ---   
 |     |     |    
   ---   ---   

   ---   ---   
 |     |     |    
 X ---   ---   
 |     |     |    
 X ---   ---   

   --- X ---   
 |     |     |    
   --- X ---   
 |     |     |    
   ---   ---   

   ---   --- X 
 |     |     |    
   ---   --- X 
 |     |     |    
   ---   ---   



### Steps to do:
- Encoder
- Decoder
- Approximate logical error rate

### Steps for gauge insertion:
- Pick a gauge operator
- remove anticommutators from stabilizer grop
- for each s in stabs: if weight(g*s) < weight(s): s = g*s